서버오픈 : mysql -u root -p<br>
서버닫기 : exit

# 상대모멘텀

In [408]:
import os   #경로 관련해서 사용할 때
import pandas as pd
import numpy as np
import datetime
import glob     #얘도 경로 관련

In [409]:
# 파일 목록 불러오기
files = glob.glob('./csv/*.csv')
files

['./csv/AMZN.csv', './csv/BND.csv', './csv/AAPL.csv']

In [410]:
# 빈데이터프레임 생성 컬럼만 지정 -> 나중에 피벗테이블로 만들어 날짜별 기업별 월수익률 df로 만든다.
month_last_df = pd.DataFrame(columns=["Date", "CODE", "1M_RET"])
month_last_df

,Date,CODE,1M_RET


In [411]:
# 종목 데이터프레임 생성
stock_df = pd.DataFrame(columns=["Date", "CODE", "Adj Close"])

In [412]:
# 함수 생성 : 시간으로 필터링 후 아래 3개 열만 가져옴 to sample
# + 종목코드, 기준년월, 수익률 컬럼을 추가
# 결과적으로, 전처리된 df와 기준년월 리스트(중복x)를 return!

def data_preprocessing(sample, ticker, base_date):
    sample['CODE'] = ticker # ticker는 종목코드 추가
    sample = sample[sample['Date'] >= base_date][["Date", "CODE", "Adj Close"]]
    sample.reset_index(inplace=True, drop=True)     #inplace는 기존꺼 수정      #drop=True는 기존 인덱스 삭제
    
    #기준년월
    sample["STD-YM"] = sample["Date"].map(lambda x : \
        datetime.datetime.strptime(x, "%Y-%m-%d").strftime("%Y-%m"))
        # 스트링에서 시계열형태로 : strptime
        # 시계열에서 스트링으로(추출) : strtime
    
    #기준년월의 중복 데이터를 삭제한 리스트
    ym_keys = list(sample["STD-YM"].unique())
    
    #수익률
    sample["1M_RET"] = 0.0
    
    return sample, ym_keys    #이렇게 리턴하면 함수 써서 저장할 때도 "A, B ="

In [413]:
df = pd.read_csv("./csv/AAPL.csv")
price_df, ym_keys = data_preprocessing(df, "AAPL", "2010-01-01")
print(price_df, ym_keys)

            Date  CODE   Adj Close   STD-YM  1M_RET
0     2010-01-04  AAPL   26.782711  2010-01     0.0
1     2010-01-05  AAPL   26.829010  2010-01     0.0
2     2010-01-06  AAPL   26.402260  2010-01     0.0
3     2010-01-07  AAPL   26.353460  2010-01     0.0
4     2010-01-08  AAPL   26.528664  2010-01     0.0
...          ...   ...         ...      ...     ...
2379  2019-06-18  AAPL  198.449997  2019-06     0.0
2380  2019-06-19  AAPL  197.869995  2019-06     0.0
2381  2019-06-20  AAPL  199.460007  2019-06     0.0
2382  2019-06-21  AAPL  198.779999  2019-06     0.0
2383  2019-06-24  AAPL  199.169998  2019-06     0.0

[2384 rows x 5 columns] ['2010-01', '2010-02', '2010-03', '2010-04', '2010-05', '2010-06', '2010-07', '2010-08', '2010-09', '2010-10', '2010-11', '2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12', '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06', '2012-07', '2012-08'

In [414]:
# df의 
def create_trade_book (sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes]
    book["STD-YM"] = book.index.map(lambda x : \
        datetime.datetime.strptime(x, "%Y-%m-%d")).strftime("%Y-%m")
    for i in sample_codes:
        book["p "+i] = ""
        book["r "+i] = ""
    return book

In [415]:
## trading 함수 생성
def tradings(book, s_codes):
    std_ym = ""
    buy_phase = ""
    for i in s_codes:
        print(i)
        # 종목코드의 인덱스 반복
        for j in book.index:
            if book.loc[j, "p "+i] == "" and book.shift(1).loc[j, "p "+i] == "ready " + i:
                std_ym = book.loc[j, "STD-YM"]
                buy_phase = True
            if book.loc[j, "p "+i] == "" and book.loc[j, "STD-YM"] == std_ym and buy_phase == True:
                #주문하는 파트
                book.loc[j, "p "+i] = "buy " + i
                print(j)
            
            if book.loc[j, "p "+i] == "":
                std_ym = None
                buy_phase = False
    return book


In [416]:
# 수익률 구하는 파트
def multi_returns(book, s_codes):
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}

    for i in book.index:
        for s in s_codes:
            if book.loc[i, 'p '+s] == "buy " + s and \
            book.shift(1).loc[i, "p "+s] == "ready " + s and \
            book.shift(2).loc[i, 'p '+s] == "":
                buy_dict[s] = book.loc[i, s]
                print("진입일 : ", i, '종목코드 : ', s, 'long 진입 가격 : ', buy_dict[s])
            elif book.loc[i, "p "+s] == "" and \
                book.shift(1).loc[i, "p "+s] == "buy "+s:
                sell_dict[s] = book.loc[i, s]
                #손익 계산
                rtn = (sell_dict[s] / buy_dict[s]) - 1
                book.loc[i, "r "+s] =rtn
                print("개별 청산일 : ", i, "종목코드 : ", s, "long 진입 가격 : ", \
                   buy_dict[s], "long 청산 가격 : ", sell_dict[s], "return : ", round(rtn*100, 2))
            
            if book.loc[i, "p " + s] == "":
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0

    acc_rtn = 1.0
    for i in book.index:
        rtn = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i, "p "+s] == "" and \
                book.shift(1).loc[i, "p "+s] == "buy "+s:
                count += 1  #count = count + 1
                rtn += book.loc[i, "r "+s]
        if (rtn != 0.0) and (count != 0):
            acc_rtn *= (rtn/count) + 1
            print("누적 청산일 : ", i, "청산 종목 수 : ", count, \
                "청산 수익율 : ", round((rtn/count), 4), "누적 수익율 : ", round(acc_rtn, 4))
            book.loc[i, "acc_rtn"] = acc_rtn
    print("누적 수익율 : ", round(acc_rtn, 4))

In [ ]:
for file in files:
    if os.path.isdir(file):
        print('%s <DIR>'%file)

    else:
        #경로(file)에서 종목코드(head)만 뽑기위해
        folder, name = os.path.split(file) #folder(폴더))= ./csv #name= MAZN.csv
        head, tail = os.path.splitext(name) #head(종목코드)= MAZN #tail(확장자)= .csv
        # print(folder, name, head, tail)
        
        read_df = pd.read_csv(file)
        # print(read_df.head(1))
        price_df, ym_keys = data_preprocessing(read_df, head, "2010-01-01")
        stock_df = pd.concat([stock_df, price_df[["Date", "CODE", "Adj Close"]]])

        for ym in ym_keys:
            ## 기준년월 마지막 종가 / 기준년월 첫 종가. => 이를 위해 기준년월(ym)로 필터링해줌
            # 2020-01인 행들 중에 가장 마지막에 있는 행의 Adj close값
            
            m_ret = price_df.loc[price_df[price_df["STD-YM"] == ym].index[-1], 'Adj Close'] \
                / price_df.loc[price_df[price_df["STD-YM"] == ym].index[0], 'Adj Close']
            price_df.loc[price_df['STD-YM'] == ym, ['1M_RET']] = m_ret
            
            month_last_df = month_last_df.append(\
                price_df.loc[price_df[price_df["STD-YM"] == ym].index[-1], ['Date', 'CODE', '1M_RET']])
            
            ## 기준년월 마지막 종가 / 기준년월 첫 종가
            # m_ret이라는 변수를 만들어서
            # ym = "2020-01"
            # price_df["STD-YM"] == "2020-01" -> STD-YM의 값이 2020-01이면 True 아니면 False
            # price_df[price_df["STD-YM"] == "2020-01"] -> []안의 조건식이 True 출력 False 미출력(필터)
            # price_df[price_df["STD-YM"] == "2020-01"].index[-1] -> 
            # STD-YM의 값이 2020-01인 값들 중에 가장 마지막에 있는 인덱스의 값
            # price_df.loc[index, column] -> 
            # index -> price_df[price_df["STD-YM"] == "2020-01"].index[-1]
            # column -> "Adj Close"
            # price_df.loc[price_df[price_df["STD-YM"] == "2020-01"].index[-1], 'Adj Close']
            # 결과적으로 price_df에 있는 STD-YM컬럼의 값이 
            # 2020-01인 행들 중에 가장 마지막에 있는 행의 Adj Close 값

In [418]:
month_last_df.head()

,Date,CODE,1M_RET
18,2010-01-29,AMZN,0.936595
37,2010-02-26,AMZN,0.996046
60,2010-03-31,AMZN,1.090172
81,2010-04-30,AMZN,1.040134
101,2010-05-28,AMZN,0.912503


# 2.
---

In [419]:
month_ret_df = month_last_df.pivot("Date", "CODE", "1M_RET")
month_ret_df

CODE,AAPL,AMZN,BND
Date,,,
2010-01-29,0.897435,0.936595,1.011692
2010-02-26,1.050789,0.996046,1.003909
2010-03-31,1.124456,1.090172,0.996728
2010-04-30,1.106454,1.040134,1.011023
2010-05-28,0.964445,0.912503,1.011821
...,...,...,...
2019-02-28,1.044274,1.008363,1.001883
2019-03-29,1.085615,1.065214,1.022547
2019-04-30,1.049310,1.061917,1.003719


In [420]:
# 투자종목 선택할 rank. 낮을수록 좋음
month_ret_df = month_ret_df.rank(axis=1, ascending=False, method="max", pct=True)
month_ret_df


CODE,AAPL,AMZN,BND
Date,,,
2010-01-29,1.000000,0.666667,0.333333
2010-02-26,0.333333,1.000000,0.666667
2010-03-31,0.333333,0.666667,1.000000
2010-04-30,0.333333,0.666667,1.000000
2010-05-28,0.666667,1.000000,0.333333
...,...,...,...
2019-02-28,0.333333,0.666667,1.000000
2019-03-29,0.333333,0.666667,1.000000
2019-04-30,0.666667,0.333333,1.000000


In [421]:
# 상위 40%에 드는 종목을 체크
month_ret_df = month_ret_df.where(month_ret_df < 0.4, np.nan).fillna(0)
month_ret_df[month_ret_df != 0] = 1
month_ret_df

CODE,AAPL,AMZN,BND
Date,,,
2010-01-29,0.0,0.0,1.0
2010-02-26,1.0,0.0,0.0
2010-03-31,1.0,0.0,0.0
2010-04-30,1.0,0.0,0.0
2010-05-28,0.0,0.0,1.0
...,...,...,...
2019-02-28,1.0,0.0,0.0
2019-03-29,1.0,0.0,0.0
2019-04-30,0.0,1.0,0.0


In [422]:
stock_list = list(stock_df["CODE"].unique())
stock_list

['AMZN', 'BND', 'AAPL']

In [423]:
# 비어있는 딕셔너리 형태 데이터
sig_dict = dict()
for date in month_ret_df.index:
    ticker_list = list(month_ret_df.loc[date, month_ret_df.loc[date, :] >= 1.0].index)
    # 날짜별로 종목코드 저장
    sig_dict[date] = ticker_list
stock_c_matrix = stock_df.pivot('Date', 'CODE', 'Adj Close')
book = create_trade_book(stock_c_matrix, stock_list)
book.head()

/var/folders/x3/lny9bbwx64b561c29r4y4z400000gn/T/ipykernel_99864/2606133954.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book["STD-YM"] = book.index.map(lambda x : \
/var/folders/x3/lny9bbwx64b561c29r4y4z400000gn/T/ipykernel_99864/2606133954.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book["p "+i] = ""
/var/folders/x3/lny9bbwx64b561c29r4y4z400000gn/T/ipykernel_99864/2606133954.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

CODE,AMZN,BND,AAPL,STD-YM,p AMZN,r AMZN,p BND,r BND,p AAPL,r AAPL
Date,,,,,,,,,,
2010-01-04,133.899994,60.611969,26.782711,2010-01,,,,,,
2010-01-05,134.690002,60.789135,26.829010,2010-01,,,,,,
2010-01-06,132.250000,60.766037,26.402260,2010-01,,,,,,
2010-01-07,130.000000,60.719822,26.353460,2010-01,,,,,,
2010-01-08,133.520004,60.781410,26.528664,2010-01,,,,,,


In [424]:

# sig_dict는 {일자, 종목코드} -> 이걸 각 변수로 따로 받음.
for date, values in sig_dict.items():
    for stock in values:
        book.loc[date, "p "+stock] = "ready " + stock
        
# 반복문이 처음 시작하는 경우
    # date - 2010-01-29
    # values = ["BND"] : 리스트형태
    # stock = 'BND' : 스트링형태
    # book.loc["2010-01-29", "p BND"] = "ready BND"
    
book.value_counts("p AAPL")

p AAPL
              2345
ready AAPL      39
dtype: int64

In [ ]:
stock_list

In [425]:
book = tradings(book, stock_list)

AMZN
2010-08-02
2010-08-03
2010-08-04
2010-08-05
2010-08-06
2010-08-09
2010-08-10
2010-08-11
2010-08-12
2010-08-13
2010-08-16
2010-08-17
2010-08-18
2010-08-19
2010-08-20
2010-08-23
2010-08-24
2010-08-25
2010-08-26
2010-08-27
2010-08-30
2010-09-01
2010-09-02
2010-09-03
2010-09-07
2010-09-08
2010-09-09
2010-09-10
2010-09-13
2010-09-14
2010-09-15
2010-09-16
2010-09-17
2010-09-20
2010-09-21
2010-09-22
2010-09-23
2010-09-24
2010-09-27
2010-09-28
2010-09-29
2010-10-01
2010-10-04
2010-10-05
2010-10-06
2010-10-07
2010-10-08
2010-10-11
2010-10-12
2010-10-13
2010-10-14
2010-10-15
2010-10-18
2010-10-19
2010-10-20
2010-10-21
2010-10-22
2010-10-25
2010-10-26
2010-10-27
2010-10-28
2010-11-01
2010-11-02
2010-11-03
2010-11-04
2010-11-05
2010-11-08
2010-11-09
2010-11-10
2010-11-11
2010-11-12
2010-11-15
2010-11-16
2010-11-17
2010-11-18
2010-11-19
2010-11-22
2010-11-23
2010-11-24
2010-11-26
2010-11-29
2010-12-01
2010-12-02
2010-12-03
2010-12-06
2010-12-07
2010-12-08
2010-12-09
2010-12-10
2010-12-13
2010-

In [ ]:
book.value_counts("p AAPL")

In [426]:
multi_returns(book, stock_list)

진입일 :  2010-02-01 종목코드 :  BND long 진입 가격 :  61.280487
개별 청산일 :  2010-03-01 종목코드 :  BND long 진입 가격 :  61.280487 long 청산 가격 :  61.585163 return :  0.5
진입일 :  2010-03-01 종목코드 :  AAPL long 진입 가격 :  26.154476
진입일 :  2010-06-01 종목코드 :  BND long 진입 가격 :  62.79007
개별 청산일 :  2010-06-01 종목코드 :  AAPL long 진입 가격 :  26.154476 long 청산 가격 :  32.642082 return :  24.8
진입일 :  2010-08-02 종목코드 :  AMZN long 진입 가격 :  120.07
개별 청산일 :  2010-08-02 종목코드 :  BND long 진입 가격 :  62.79007 long 청산 가격 :  64.18457 return :  2.22
개별 청산일 :  2011-02-01 종목코드 :  AMZN long 진입 가격 :  120.07 long 청산 가격 :  172.110001 return :  43.34
진입일 :  2011-02-01 종목코드 :  AAPL long 진입 가격 :  43.179466
진입일 :  2011-04-01 종목코드 :  AMZN long 진입 가격 :  180.130005
개별 청산일 :  2011-04-01 종목코드 :  AAPL long 진입 가격 :  43.179466 long 청산 가격 :  43.120655 return :  -0.14
개별 청산일 :  2011-06-01 종목코드 :  AMZN long 진입 가격 :  180.130005 long 청산 가격 :  192.399994 return :  6.81
진입일 :  2011-06-01 종목코드 :  BND long 진입 가격 :  66.480408
진입일 :  2011-07-01 종목코드 :  AMZN long 진입 가격 

/var/folders/x3/lny9bbwx64b561c29r4y4z400000gn/T/ipykernel_99864/1510890031.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book.loc[i, "acc_rtn"] = acc_rtn


 2011-06-01 청산 종목 수 :  1 청산 수익율 :  0.0681 누적 수익율 :  1.9603
누적 청산일 :  2011-07-01 청산 종목 수 :  1 청산 수익율 :  -0.0081 누적 수익율 :  1.9444
누적 청산일 :  2011-08-01 청산 종목 수 :  1 청산 수익율 :  0.0565 누적 수익율 :  2.0542
누적 청산일 :  2011-09-01 청산 종목 수 :  1 청산 수익율 :  -0.0396 누적 수익율 :  1.9728
누적 청산일 :  2011-10-03 청산 종목 수 :  1 청산 수익율 :  0.0081 누적 수익율 :  1.9888
누적 청산일 :  2011-11-01 청산 종목 수 :  1 청산 수익율 :  0.0006 누적 수익율 :  1.9899
누적 청산일 :  2011-12-01 청산 종목 수 :  1 청산 수익율 :  -0.0216 누적 수익율 :  1.9469
누적 청산일 :  2012-01-03 청산 종목 수 :  1 청산 수익율 :  0.009 누적 수익율 :  1.9643
누적 청산일 :  2012-04-02 청산 종목 수 :  1 청산 수익율 :  0.5043 누적 수익율 :  2.955
누적 청산일 :  2012-06-01 청산 종목 수 :  1 청산 수익율 :  0.0514 누적 수익율 :  3.1068
누적 청산일 :  2012-07-02 청산 종목 수 :  1 청산 수익율 :  -0.0007 누적 수익율 :  3.1045
누적 청산일 :  2012-08-01 청산 종목 수 :  1 청산 수익율 :  0.0121 누적 수익율 :  3.142
누적 청산일 :  2012-10-01 청산 종목 수 :  1 청산 수익율 :  0.0913 누적 수익율 :  3.4289
누적 청산일 :  2012-11-01 청산 종목 수 :  1 청산 수익율 :  -0.0788 누적 수익율 :  3.1585
누적 청산일 :  2012-12-03 청산 종목 수 :  1 청산 수익율 :  0.0029 누적 수

In [427]:
book.tail(50)

CODE,AMZN,BND,AAPL,STD-YM,p AMZN,r AMZN,p BND,r BND,p AAPL,r AAPL,acc_rtn
Date,,,,,,,,,,,
2019-04-12,1843.060059,80.019890,198.107086,2019-04,,,,,buy AAPL,,NaN
2019-04-15,1844.869995,80.039757,198.465714,2019-04,,,,,buy AAPL,,NaN
2019-04-16,1863.040039,79.900749,198.485641,2019-04,,,,,buy AAPL,,NaN
2019-04-17,1864.819946,79.900749,202.350754,2019-04,,,,,buy AAPL,,NaN
2019-04-18,1861.689941,80.009972,203.077957,2019-04,,,,,buy AAPL,,NaN
2019-04-22,1887.310059,79.930542,203.745377,2019-04,,,,,buy AAPL,,NaN
2019-04-23,1923.770020,80.069550,206.684052,2019-04,,,,,buy AAPL,,NaN
2019-04-24,1901.750000,80.278061,206.365295,2019-04,,,,,buy AAPL,,NaN
2019-04-25,1902.250000,80.258209,204.492508,2019-04,,,,,buy AAPL,,NaN
